# Retrieval Augemented Generation (RAG)

In the prevous two recepies, we learned how to [obtain an embedding](./10-embeddings.ipynb), as well as how to find the [similarity](./11-similarity-search.ipynb) between two embeddings. Retrievel Augmente Generation uses both of these to achieve the goal of querying an LLM and also providing some documents for additional context. For example, if you want the LLM to answer some questions based on specific documents you have, you can feed those documents in 

A RAG model is primarily made up of two components, the retriever, and the reader

In [3]:
from langchain.docstore.document import Document
from langchain_dartmouth.retrievers.document_compressors import DartmouthReranker
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

Given a **collection** of documents, we are interested in finding the ones most relevent to our question. Once a query is posed, the `compress_documents()` function from the `DartmouthReranker` class can be used, which uses the concept about [similarity](./11-similarity-search.ipynb), to return a list of documents which are most relevant to the query.

### Restructure the RAG a bit 
    - Ramp things up in complexity
    - Simplest way of a RAG is loading and then stuffing the prompt with a text. 
    - Next thing is, what if you have multiple documents? And you want the most relevant one? Reranking would be here. If you have a few and want to pull out all of them. 
    - 

### A Simple RAG

At it's heart a RAG is akin to just pluging in an entire chunk of text that we want the LLM to reference when in it's answer. An example of this can be found below

In [4]:
from langchain_dartmouth.llms import ChatDartmouth
llm = ChatDartmouth(model_name="llama-3-1-8b-instruct")

relevant_document = "Asteroids do not generally hit people. There is a 1 in a 7 billion chance for that to happen"
query = "Are asteroids going to hit me?"

response = llm.invoke(relevant_document + 'Considering this, answer the following question:' +  query)
print(response.content)

Based on the information provided, the chances of an asteroid hitting you are extremely low, approximately 1 in 7 billion. This means that the likelihood of an asteroid impacting you is virtually negligible.

So, to answer your question: no, it is highly unlikely that an asteroid is going to hit you.


### Reranking Documents

That was simple! What if we have many documents? In that case we can use the `DartmouthReranker` to find which document is the most relevant. 

<div class="alert alert-info">

**Note:** The `DirectoryLoader` function is a part of LangChain that accepts a directory, a regex expression, and a [loader class](./10-embeddings.ipynb). It's an easy way to load several documents that are in a directory at once
</div>



In [5]:
# Load the documents
loader = DirectoryLoader('./rag_documents', glob="**/*.txt", loader_cls=TextLoader)
collection = loader.load()

In [6]:
reranker = DartmouthReranker()

query = "Are asteroids going to hit me?"

ranked_docs = reranker.compress_documents(query=query, documents=collection)

for doc in ranked_docs:
    print(doc.metadata['source'])

rag_documents\asteroids.txt
rag_documents\hot_sauce.txt
rag_documents\history.txt


We can see that when our query is related to asteroids, the document correcly ranks the asteroids.txt as the most relevent document. Now, we can prepend the content of the first ranked document to our query to get a response which considers the information in the document

In [7]:
from langchain_dartmouth.llms import ChatDartmouth

llm = ChatDartmouth(model_name="llama-3-1-8b-instruct")
response = llm.invoke(ranked_docs[0].page_content + 'Considering this, answer the following question:' +  query)
print(response.content)

The chances of an asteroid hitting you are extremely low. However, it's not impossible. The likelihood of being hit by an asteroid depends on various factors, such as the size and frequency of asteroid impacts, as well as your location on Earth.

According to NASA, there are over 18,000 known near-Earth asteroids (NEAs), and many more are awaiting discovery. However, the vast majority of these asteroids are small, measuring less than 100 feet in diameter, and they burn up in the atmosphere or disintegrate before hitting the ground.

To put the risk into perspective:

1. **The odds of being hit by a meteorite (a small piece of an asteroid) are about 1 in 1.9 million.**
2. **The likelihood of being hit by a large asteroid (one that could cause significant damage) is estimated to be about 1 in 1.4 million per year.** However, this risk is spread over the entire population of the Earth, making the chances of being hit by an asteroid extremely low.

To further reduce the risk, scientists an

## Using Vector Stores
There is a useful feature in Langchain called `vector stores`. This allows us to store several documents in one go. The best part of this is that we don't need to prepend an entire document. We can break the documents into smaller chunks and only use the most relevent parts. 

In [31]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_dartmouth.embeddings import DartmouthEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# Initialize the text splitter with appropriate chunk size
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200,length_function=len)

# Load and split the documents
collection = loader.load()
chunks = text_splitter.split_documents(collection)

# Initialize vector store and add chunks
vector_store = InMemoryVectorStore(embedding=DartmouthEmbeddings())
vector_store.add_documents(chunks)
pass

In [33]:
docs = vector_store.similarity_search(query)
docs

[Document(id='e4d85fe6-897e-477c-80f5-420039ecfb94', metadata={'source': 'rag_documents\\asteroids.txt'}, page_content='**Asteroids: The Mysterious and Ancient Building Blocks of Our Solar System**\nAsteroids, also known as minor planets or planetoids, are small, rocky objects that orbit the Sun. They are remnants from the early days of our solar system, and their study has provided valuable insights into the formation and evolution of the cosmos. These mysterious bodies have captivated the imagination of scientists and researchers for centuries, and their exploration continues to uncover new secrets about the universe.\n**Composition and Types of Asteroids**\nAsteroids are typically small, with diameters ranging from a few meters to hundreds of kilometers. They are composed of rock, metal, and ice, and are thought to be the remnants of the early solar system. There are two main types of asteroids: stony asteroids, which are composed mostly of silicate minerals, and metal asteroids, wh

# Summary
In this recipe we use a query and `DartmouthReranker` to retrieve relative documents from a **collection**. The content of the highest ranked document is then prepended in the a prompt to an llm. This is an implementation of retreival augmented generation.